In [11]:
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install datasets evaluate tensorboard transformers accelerate kagglehub sentencepiece


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import torch
import pandas as pd
import numpy as np
import re
from matplotlib import pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification

from datasets import Dataset
from datasets import concatenate_datasets
from datasets import DatasetDict
from datasets import ClassLabel

import os
os.environ["KAGGLE_USERNAME"] = "USERNAME"
os.environ["KAGGLE_KEY"] = "KAGGLE_KEY"
import kagglehub

In [2]:
# Download the latest version.
dataset_path = kagglehub.dataset_download("abdoukarimkandji/wolbanking77v1")
print(dataset_path)

100%|██████████| 1.02M/1.02M [00:00<00:00, 1.29MB/s]

Extracting files...
/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/4


In [5]:
!ls /root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/4/wolbanking77v1/full

test  train


In [2]:
# Set the Seed for Reproducible Results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
df_train = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/4/wolbanking77v1/full/train/train.csv")

df_test = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/4/wolbanking77v1/full/test/test.csv")

df_train = df_train[['input_wo', 'label']]
df_test = df_test[['input_wo', 'label']]

#removal of punctuation marks
df_train['input_wo_clean'] = df_train['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')
df_test['input_wo_clean'] = df_test['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')

# Apply lowercase
df_train['input_wo_clean'] = df_train['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['input_wo_clean'] = df_test['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))

train_ds = Dataset.from_pandas(df_train[['input_wo_clean', 'label']], split="train")
test_ds = Dataset.from_pandas(df_test[['input_wo_clean', 'label']], split="test")

raw_dataset = DatasetDict({"train": train_ds, "test": test_ds})
# raw_dataset = raw_dataset.remove_columns("__index_level_0__")
raw_dataset =  raw_dataset.rename_column("input_wo_clean", "text") # to match Trainer
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7832
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1959
    })
})

In [5]:
# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt")
    # return tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")

In [6]:
from transformers import AutoTokenizer
from transformers import XLMRobertaModel, XLMRobertaTokenizer

# Model id to load the tokenizer
model_id = "bonadossou/afrolm_active_learning"
# Load Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_id)
tokenizer.model_max_length = 256

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/6.01M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

In [7]:
# tokenize(raw_dataset['train'])
tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/7832 [00:00<?, ? examples/s]

Map:   0%|          | 0/1959 [00:00<?, ? examples/s]

In [8]:
labels = []
for label in tokenized_dataset["train"]:
    labels.append(label['label'])
labels = list(set(labels))
len(labels)

77

In [9]:
new_features = tokenized_dataset['train'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['train'] = tokenized_dataset['train'].cast(new_features)
new_features = tokenized_dataset['test'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['test'] = tokenized_dataset['test'].cast(new_features)

Casting the dataset:   0%|          | 0/7832 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1959 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSequenceClassification

# Prepare model labels - useful for inference
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label)

pytorch_model.bin:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at bonadossou/afrolm_active_learning and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

In [11]:
import evaluate
import numpy as np

# Metric Id
metric = evaluate.load("f1")

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [ ]:
#del trainer
#torch.cuda.empty_cache()

In [12]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Id for remote repository
repository_id = "afrolm_active_learning-banking77-full-wolof"

# Define training args
training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=16, # 8 16
    per_device_eval_batch_size=8, # 8
    learning_rate=2e-05,
	num_train_epochs=20, # 5
    warmup_ratio=0.1,
    weight_decay=0.01,
		# PyTorch 2.0 specifics 
    # bf16=True, # bfloat16 training 
	# torch_compile=True, # optimizations
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    remove_unused_columns=False

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [13]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,4.305100,4.120271,0.025656
2,3.890400,3.420973,0.171871
3,2.975600,2.799716,0.358085
4,2.512900,2.495224,0.454749
5,2.095400,2.319457,0.506618
6,1.856100,2.269553,0.525133
7,1.614900,2.207303,0.551608
8,1.450600,2.205044,0.558747
9,1.295200,2.230413,0.557659
10,1.127300,2.251700,0.563091


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 256}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters:

TrainOutput(global_step=9800, training_loss=1.4593597505530533, metrics={'train_runtime': 6127.2695, 'train_samples_per_second': 25.564, 'train_steps_per_second': 1.599, 'total_flos': 1.721006400774144e+16, 'train_loss': 1.4593597505530533, 'epoch': 20.0})

In [14]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 2.2835943698883057,
 'eval_f1': 0.5687058970499536,
 'eval_runtime': 14.2149,
 'eval_samples_per_second': 137.813,
 'eval_steps_per_second': 17.235,
 'epoch': 20.0}

In [15]:
# Save processor and create model card
tokenizer.save_pretrained(repository_id)

('afrolm_active_learning-banking77-full-wolof/tokenizer_config.json',
 'afrolm_active_learning-banking77-full-wolof/special_tokens_map.json',
 'afrolm_active_learning-banking77-full-wolof/sentencepiece.bpe.model',
 'afrolm_active_learning-banking77-full-wolof/added_tokens.json')

In [17]:
model_id = "/workspace/NLP_TASKS/afrolm_active_learning-banking77-full-wolof/checkpoint-9800"
# tokenizer = AutoTokenizer.from_pretrained(repository_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, use_safetensors=True).to("cuda")
tokenizer = XLMRobertaTokenizer.from_pretrained(repository_id)
tokenizer.model_max_length = 256

In [18]:
def predict(batch, model):
    input_ids = torch.tensor(batch['input_ids']).to(device).unsqueeze(0)
    # token_type_ids = torch.tensor(batch['token_type_ids']).to(device).unsqueeze(0)
    attention_mask = torch.tensor(batch['attention_mask']).to(device).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_ids=input_ids, 
                       # token_type_ids=token_type_ids, 
                       attention_mask=attention_mask).logits
        predicted = torch.argmax(logits, dim=-1)[0]
    return predicted

In [19]:
tokenized_dataset["test"][0].keys()

dict_keys(['label', 'input_ids', 'attention_mask'])

In [20]:
model.to("cuda")
predictions = [predict(batch, model) for batch in tokenized_dataset["test"]]

In [21]:
predictions[0]

tensor(15, device='cuda:0')

In [22]:
y_pred = []
for y in predictions:
    y_pred.append(labels[y])
y_pred

['Refund not showing up',
 'transfer into account',
 'card payment fee charged',
 'direct debit payment not recognised',
 'getting spare card',
 'pending card payment',
 'request refund',
 'receiving money',
 'top up by card charge',
 'why verify identity',
 'cash withdrawal not recognised',
 'card acceptance',
 'declined card payment',
 'transaction charged twice',
 'card payment fee charged',
 'exchange via app',
 'pending transfer',
 'card about to expire',
 'lost or stolen card',
 'direct debit payment not recognised',
 'pending top up',
 'top up reverted',
 'pending cash withdrawal',
 'exchange rate',
 'apple pay or google pay',
 'balance not updated after bank transfer',
 'cash withdrawal not recognised',
 'pending cash withdrawal',
 'fiat currency support',
 'cash withdrawal charge',
 'transfer timing',
 'card about to expire',
 'cancel transfer',
 'apple pay or google pay',
 'declined transfer',
 'wrong exchange rate for cash withdrawal',
 'failed transfer',
 'transfer timing',

In [23]:
y_test = []
for y in tokenized_dataset["test"]['label']:
    y_test.append(labels[y])
y_test

['activate my card',
 'transfer into account',
 'card payment fee charged',
 'unable to verify identity',
 'getting spare card',
 'pending card payment',
 'request refund',
 'receiving money',
 'top up by card charge',
 'why verify identity',
 'cash withdrawal not recognised',
 'visa or mastercard',
 'card not working',
 'transaction charged twice',
 'card payment fee charged',
 'exchange via app',
 'getting spare card',
 'card acceptance',
 'lost or stolen card',
 'request refund',
 'pending top up',
 'topping up by card',
 'pending cash withdrawal',
 'declined card payment',
 'apple pay or google pay',
 'top up failed',
 'direct debit payment not recognised',
 'pending cash withdrawal',
 'supported cards and currencies',
 'cash withdrawal charge',
 'virtual card not working',
 'card about to expire',
 'cancel transfer',
 'apple pay or google pay',
 'declined transfer',
 'verify my identity',
 'card not working',
 'balance not updated after cheque or cash deposit',
 'extra charge on s

In [24]:
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.53      0.48      0.51        33
                                activate my card       0.56      0.47      0.51        32
                                       age limit       0.60      0.79      0.68        19
                         apple pay or google pay       0.65      0.68      0.67        25
                                     atm support       0.55      0.80      0.65        15
                                automatic top up       0.62      0.62      0.62        24
         balance not updated after bank transfer       0.53      0.56      0.54        34
balance not updated after cheque or cash deposit       0.65      0.61      0.63        36
                         beneficiary not allowed       0.34      0.35      0.35        31
                                 cancel transfer       0.64      0.70      0.67        30
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.5637473864729473

In [26]:
metrics.accuracy_score(y_test, y_pred)

0.5666156202143952

In [27]:
metrics.precision_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.5701129610226796

In [28]:
metrics.recall_score(y_test, y_pred, average='weighted')

0.5666156202143952

In [29]:
def compute_metrics(y_true, y_pred):
    print("f1_score:", metrics.f1_score(y_true, y_pred, average='weighted'))
    print("precision_score:", metrics.precision_score(y_true, y_pred, average='weighted'))
    print("recall_score:", metrics.recall_score(y_true, y_pred, average='weighted'))

In [30]:
compute_metrics(y_test, y_pred)

f1_score: 0.5637473864729473
precision_score: 0.5701129610226796
recall_score: 0.5666156202143952


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
